# UxC price exploration (Plotly)

Interactive visuals across daily, weekly, and month-end UxC data. Cells degrade gracefully when inputs are missing.


In [ ]:
import sys
from pathlib import Path

# Ensure repo src is on path before importing project modules
try:
    REPO_ROOT = Path(__file__).resolve().parents[1]
except NameError:
    cwd = Path.cwd().resolve()
    REPO_ROOT = cwd if (cwd / "src").exists() else cwd.parent

if str(REPO_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / "src"))

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from uranium_model.connections.postgres import get_engine, test_connection
from uranium_model.data.uxc import build_price_features, load_all_uxc_prices, to_annual, to_monthly

pio.templates.default = "plotly_white"
PALETTE = {
    "spot": "#0d6efd",
    "spot_weekly": "#3e5ccc",
    "spot_daily": "#7da2f2",
    "forward": "#d62f64",
    "forward5": "#9c3fa6",
    "forward_lt": "#2ca02c",
    "basis_na": "#ff7f0e",
    "basis_eu": "#1f77b4",
    "swu_spot": "#2ca02c",
    "swu_lt": "#d62728",
    "uf6_na": "#17becf",
    "uf6_eu": "#9467bd",
}

pd.options.display.float_format = "{:,.2f}".format


In [ ]:
# Configure date window
START = "2005-01-01"  # set None to pull everything
END = None
HIGHLIGHT_LAST_DAYS = 365


In [ ]:
# Load all three tables with graceful fallbacks
engine = get_engine()
ok, err = test_connection(engine)
if not ok:
    raise RuntimeError(f"Chrono connection failed: {err}")

frames = load_all_uxc_prices(engine, start=START, end=END)

daily = frames.get("daily", pd.DataFrame())
weekly = frames.get("weekly", pd.DataFrame())
month_end = frames.get("month_end", pd.DataFrame())

monthly = month_end if not month_end.empty else to_monthly(daily)
weekly_monthly = to_monthly(weekly) if not weekly.empty else pd.DataFrame()
annual = to_annual(monthly) if not monthly.empty else pd.DataFrame()
annual_features = build_price_features(annual) if not annual.empty else annual

def _cols(df: pd.DataFrame) -> str:
    if df is None or df.empty:
        return "none"
    return ", ".join(sorted(df.columns))

print("Loaded frames (rows, cols):")
for name, df in frames.items():
    print(f"  {name:<9}: {len(df):>6} rows | columns: {_cols(df)}")
print(f"Monthly frame: {len(monthly)} rows (from {'month_end' if not month_end.empty else 'daily avg'})")


def has(df: pd.DataFrame, cols) -> bool:
    cols_set = set(cols)
    return not df.empty and cols_set.issubset(df.columns)


## Cross-frequency spot levels
Layered spot across month-end, weekly, and smoothed daily prints. Last-year window shaded.


In [ ]:
spot_fig = go.Figure()

if has(month_end, ["u3o8_spot"]):
    spot_fig.add_scatter(
        x=month_end.index,
        y=month_end["u3o8_spot"],
        name="Spot (month end)",
        line=dict(color=PALETTE["spot"], width=2.2),
    )

if has(weekly, ["u3o8_spot"]):
    spot_fig.add_scatter(
        x=weekly.index,
        y=weekly["u3o8_spot"],
        name="Spot (weekly)",
        line=dict(color=PALETTE["spot_weekly"], width=1.6, dash="dot"),
    )

if has(daily, ["u3o8_spot"]):
    daily_smooth = daily["u3o8_spot"].rolling("14D").mean()
    spot_fig.add_scatter(
        x=daily_smooth.index,
        y=daily_smooth,
        name="Spot (daily 14d avg)",
        line=dict(color=PALETTE["spot_daily"], width=1.2),
        opacity=0.75,
    )

if not spot_fig.data:
    print("No spot columns available to plot.")
else:
    combined_index = pd.concat(
        [df[["u3o8_spot"]] for df in (month_end, weekly, daily) if has(df, ["u3o8_spot"])]
    ).index
    if not combined_index.empty and HIGHLIGHT_LAST_DAYS:
        end_date = combined_index.max()
        start_date = end_date - pd.Timedelta(days=HIGHLIGHT_LAST_DAYS)
        spot_fig.add_vrect(x0=start_date, x1=end_date, fillcolor="rgba(13,110,253,0.08)", line_width=0)
    spot_fig.update_layout(
        title="U3O8 spot across frequencies",
        yaxis_title="USD/lb",
        hovermode="x unified",
        legend_title="Series",
    )
    spot_fig.update_xaxes(rangeslider_visible=True)
    spot_fig.show()


## Forward curve time-lapse (animation)
Watch how the term structure morphs over time.


In [ ]:
fc_source = month_end if has(month_end, ["u3o8_spot"]) else weekly_monthly

if fc_source.empty or "u3o8_spot" not in fc_source.columns:
    print("No forward data available for animation.")
else:
    tenor_map = [
        ("Spot", 0, "u3o8_spot"),
        ("3y", 3, "yr_3_fwd_u3o8"),
        ("5y", 5, "yr_5_fwd_u3o8"),
        ("LT", 10, "lt_u3o8"),
    ]
    rows = []
    for dt, row in fc_source.iterrows():
        for tenor, years, col in tenor_map:
            if col in fc_source.columns and pd.notna(row.get(col)):
                rows.append({"date": dt, "frame": dt.strftime("%Y-%m"), "tenor": tenor, "years": years, "price": row[col]})
    if not rows:
        print("Forward columns missing for animation.")
    else:
        long_df = pd.DataFrame(rows)
        y_min, y_max = long_df["price"].min(), long_df["price"].max()
        padding = (y_max - y_min) * 0.1 if y_max != y_min else 1
        fig = px.line(
            long_df,
            x="years",
            y="price",
            color="tenor",
            animation_frame="frame",
            markers=True,
            title="Forward curve time-lapse",
            range_x=[-0.2, 11],
            range_y=[y_min - padding, y_max + padding],
        )
        fig.update_layout(hovermode="x", legend_title="Tenor")
        fig.show()


## Forward curves (static overlay)
Uses month-end prints when available; falls back to weekly averages.


In [ ]:
source = month_end if has(month_end, ["u3o8_spot"]) else weekly_monthly
if source.empty or "u3o8_spot" not in source.columns:
    print("No forward/spot columns available for curves.")
else:
    term_cols = [c for c in ["yr_3_fwd_u3o8", "yr_5_fwd_u3o8", "lt_u3o8"] if c in source.columns]
    if not term_cols:
        print("No forward columns found in source table.")
    else:
        fig = go.Figure()
        fig.add_scatter(x=source.index, y=source["u3o8_spot"], name="Spot", line=dict(color=PALETTE["spot"], width=2.2))
        colors = [PALETTE["forward"], PALETTE["forward5"], PALETTE["forward_lt"]]
        for idx, col in enumerate(term_cols):
            fig.add_scatter(
                x=source.index,
                y=source[col],
                name=col.replace("_", " "),
                line=dict(color=colors[idx % len(colors)], width=1.8, dash="dot"),
            )
        fig.update_layout(
            title="U3O8 forward curves vs spot",
            yaxis_title="USD/lb",
            hovermode="x unified",
        )
        fig.show()


## Forward premia/discount vs spot (area)
Zero line highlighted; compares 3y/5y/LT premia.


In [ ]:
spread_source = month_end if has(month_end, ["u3o8_spot"]) else weekly_monthly
spread_map = {}
if has(spread_source, ["yr_3_fwd_u3o8", "u3o8_spot"]):
    spread_map["3y"] = spread_source["yr_3_fwd_u3o8"] - spread_source["u3o8_spot"]
if has(spread_source, ["yr_5_fwd_u3o8", "u3o8_spot"]):
    spread_map["5y"] = spread_source["yr_5_fwd_u3o8"] - spread_source["u3o8_spot"]
if has(spread_source, ["lt_u3o8", "u3o8_spot"]):
    spread_map["LT"] = spread_source["lt_u3o8"] - spread_source["u3o8_spot"]

if not spread_map:
    print("No forward spread columns available.")
else:
    fig = go.Figure()
    color_cycle = ["#d62f64", "#9c3fa6", "#2ca02c"]
    for idx, (name, series) in enumerate(spread_map.items()):
        fig.add_scatter(
            x=series.index,
            y=series,
            name=name,
            mode="lines",
            line=dict(color=color_cycle[idx % len(color_cycle)], width=1.8),
            fill="tozeroy",
            fillcolor=f"rgba(0,0,0,{0.06 * (idx+1)})",
        )
    fig.add_hline(y=0, line=dict(color="gray", width=1, dash="dash"))
    fig.update_layout(title="Forward premia/discount vs spot (USD/lb)", yaxis_title="USD/lb", hovermode="x unified")
    fig.show()


## Forward premia heatmap
Normalized view of term premia vs spot.


In [ ]:
spread_source = month_end if has(month_end, ["u3o8_spot"]) else weekly_monthly
spread_map = {}
if has(spread_source, ["yr_3_fwd_u3o8", "u3o8_spot"]):
    spread_map["3y"] = spread_source["yr_3_fwd_u3o8"] - spread_source["u3o8_spot"]
if has(spread_source, ["yr_5_fwd_u3o8", "u3o8_spot"]):
    spread_map["5y"] = spread_source["yr_5_fwd_u3o8"] - spread_source["u3o8_spot"]
if has(spread_source, ["lt_u3o8", "u3o8_spot"]):
    spread_map["LT"] = spread_source["lt_u3o8"] - spread_source["u3o8_spot"]

if not spread_map:
    print("No forward columns available for heatmap.")
else:
    spreads = pd.DataFrame(spread_map, index=spread_source.index)
    fig = px.imshow(
        spreads.T,
        aspect="auto",
        color_continuous_scale="RdBu_r",
        origin="lower",
        labels=dict(color="USD/lb"),
        title="Forward premia/discount vs spot (USD/lb)",
        text_auto=".1f",
    )
    fig.update_layout(height=360)
    fig.show()


## Daily detail (MAP, DAP, spot)
Focus on near-term indicators with smoothing overlays.


In [ ]:
if daily.empty:
    print("Daily table empty; skipping MAP/DAP view.")
else:
    series_specs = [
        ("u3o8_spot", "Spot", PALETTE["spot"], 2.0),
        ("u3o8_map_d", "MAP", "#f59e0b", 1.6),
        ("u3o8_dap", "DAP", "#6c757d", 1.4),
    ]
    fig = go.Figure()
    added = False
    for col, label, color, width in series_specs:
        if col in daily.columns:
            fig.add_scatter(
                x=daily.index,
                y=daily[col],
                name=label,
                mode="lines",
                line=dict(color=color, width=width),
                opacity=0.85,
            )
            smooth = daily[col].rolling("30D").mean()
            fig.add_scatter(
                x=smooth.index,
                y=smooth,
                name=f"{label} (30d avg)",
                line=dict(color=color, width=1.0, dash="dash"),
                opacity=0.6,
            )
            added = True
    if not added:
        print("MAP/DAP/spot columns missing from daily table.")
    else:
        fig.update_layout(
            title="Daily detail (MAP / DAP / Spot)",
            yaxis_title="USD/lb",
            hovermode="x unified",
            legend_orientation="h",
            legend=dict(y=-0.25),
        )
        fig.update_xaxes(rangeslider_visible=True)
        fig.show()


## Conversion & SWU
Two-pane view: NA/EU conversion basis and SWU spot vs LT.


In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08, subplot_titles=("Conversion basis (spot - LT)", "SWU spot vs LT"))

basis_plotted = False
if has(month_end, ["na_conv", "na_lt_conv"]):
    fig.add_scatter(x=month_end.index, y=month_end["na_conv"] - month_end["na_lt_conv"], name="NA basis", line=dict(color=PALETTE["basis_na"], width=1.8), row=1, col=1)
    basis_plotted = True
if has(month_end, ["eu_conv", "eu_lt_conv"]):
    fig.add_scatter(x=month_end.index, y=month_end["eu_conv"] - month_end["eu_lt_conv"], name="EU basis", line=dict(color=PALETTE["basis_eu"], width=1.8, dash="dot"), row=1, col=1)
    basis_plotted = True
if basis_plotted:
    fig.add_hline(y=0, line=dict(color="gray", width=1, dash="dash"), row=1, col=1)
else:
    fig.add_annotation(text="No conversion columns available", xref="paper", yref="paper", x=0.01, y=0.75, showarrow=False)

swu_plotted = False
if has(month_end, ["spot_swu"]):
    fig.add_scatter(x=month_end.index, y=month_end["spot_swu"], name="SWU spot", line=dict(color=PALETTE["swu_spot"], width=1.8), row=2, col=1)
    swu_plotted = True
if has(month_end, ["lt_swu"]):
    fig.add_scatter(x=month_end.index, y=month_end["lt_swu"], name="SWU LT", line=dict(color=PALETTE["swu_lt"], width=1.8, dash="dot"), row=2, col=1)
    swu_plotted = True
if swu_plotted:
    fig.add_hline(y=0, line=dict(color="gray", width=1, dash="dash"), row=2, col=1)
else:
    fig.add_annotation(text="No SWU columns available", xref="paper", yref="paper", x=0.01, y=0.25, showarrow=False)

fig.update_layout(height=650, hovermode="x unified")
fig.update_yaxes(title_text="USD/kgU", row=1, col=1)
fig.update_yaxes(title_text="USD/SWU", row=2, col=1)
fig.show()


## UF6 vs U3O8 value
Monthly overlay of UF6 values vs U3O8 spot for NA/EU.


In [ ]:
if month_end.empty:
    print("Month-end table empty; skipping UF6 view.")
else:
    has_na = has(month_end, ["na_uf6_value", "u3o8_spot"])
    has_eu = has(month_end, ["eu_uf6_value", "u3o8_spot"])
    if not (has_na or has_eu):
        print("UF6 columns missing from month_end.")
    else:
        fig = go.Figure()
        if has_na:
            fig.add_scatter(
                x=month_end.index,
                y=month_end["na_uf6_value"],
                name="NA UF6 value",
                line=dict(color=PALETTE["uf6_na"], width=1.8),
            )
        if has_eu:
            fig.add_scatter(
                x=month_end.index,
                y=month_end["eu_uf6_value"],
                name="EU UF6 value",
                line=dict(color=PALETTE["uf6_eu"], width=1.8, dash="dot"),
            )
        fig.add_scatter(
            x=month_end.index,
            y=month_end["u3o8_spot"],
            name="U3O8 spot",
            line=dict(color=PALETTE["spot"], width=1.4),
            opacity=0.7,
        )
        fig.update_layout(title="UF6 value vs U3O8 spot (month end)", yaxis_title="USD/unit", hovermode="x unified")
        fig.show()


## Volatility & drawdowns
Adaptive window by frequency plus drawdown depth band.


In [ ]:
if has(daily, ["u3o8_spot"]):
    vol_source = daily
    window = 63
    annualizer = 252 ** 0.5
    freq_used = "daily"
elif has(weekly, ["u3o8_spot"]):
    vol_source = weekly
    window = 26
    annualizer = 52 ** 0.5
    freq_used = "weekly"
elif has(monthly, ["u3o8_spot"]):
    vol_source = monthly
    window = 12
    annualizer = 12 ** 0.5
    freq_used = "monthly"
else:
    vol_source = pd.DataFrame()
    window = 0
    annualizer = 1
    freq_used = None

if vol_source.empty:
    print("No spot series to compute volatility.")
else:
    ret = vol_source["u3o8_spot"].pct_change()
    vol = ret.rolling(window=window).std() * annualizer
    spot_series = vol_source["u3o8_spot"]
    roll_max = spot_series.cummax()
    drawdown = (spot_series / roll_max) - 1

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        specs=[[{"secondary_y": False}], [{"secondary_y": False}]],
        subplot_titles=(f"Rolling vol ({freq_used}, window={window})", "Spot drawdown from peak"),
    )

    fig.add_scatter(x=vol.index, y=vol, name="Vol", line=dict(color="#17becf", width=2.2), row=1, col=1)
    fig.update_yaxes(title_text="Annualized vol", row=1, col=1)

    fig.add_scatter(
        x=drawdown.index,
        y=drawdown,
        name="Drawdown",
        mode="lines",
        line=dict(color="#d62f64", width=1.8),
        fill="tozeroy",
        fillcolor="rgba(214,47,100,0.2)",
        row=2,
        col=1,
    )
    fig.add_hline(y=0, line=dict(color="gray", width=1, dash="dash"), row=2, col=1)
    fig.update_yaxes(title_text="Pct off high", tickformat=".0%", row=2, col=1)

    fig.update_layout(height=650, hovermode="x unified")
    fig.show()


## Spot vs forward premia scatter
Helps see how premia co-move with absolute price. Trendlines included.


In [ ]:
spread_source = month_end if has(month_end, ["u3o8_spot"]) else weekly_monthly
spread_map = {}
if has(spread_source, ["yr_3_fwd_u3o8", "u3o8_spot"]):
    spread_map["3y"] = spread_source["yr_3_fwd_u3o8"] - spread_source["u3o8_spot"]
if has(spread_source, ["yr_5_fwd_u3o8", "u3o8_spot"]):
    spread_map["5y"] = spread_source["yr_5_fwd_u3o8"] - spread_source["u3o8_spot"]
if has(spread_source, ["lt_u3o8", "u3o8_spot"]):
    spread_map["LT"] = spread_source["lt_u3o8"] - spread_source["u3o8_spot"]

if not spread_map:
    print("No forward spreads to plot.")
else:
    df = pd.DataFrame(spread_map)
    df["spot"] = spread_source["u3o8_spot"]
    df["year"] = df.index.year
    melted = df.reset_index().melt(id_vars=["index", "spot", "year"], value_name="spread", var_name="tenor")
    fig = px.scatter(
        melted,
        x="spot",
        y="spread",
        color="tenor",
        trendline="ols",
        hover_data={"index": True, "year": True, "tenor": True},
        labels={"spot": "Spot (USD/lb)", "spread": "Premium (USD/lb)"},
        title="Spot vs forward premia",
    )
    fig.show()


## Annual features (preview)
Quick peek at derived features used in regressions.


In [ ]:
annual_features.tail(10)
